<a href="https://colab.research.google.com/github/RafaelBNN/if697/blob/main/Projeto_2_IF697.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A ideia vai ser prever o `Place` de um atleta de acordo com `Age`, `BodyweightKg`, `Sex`, `Division`, `IPFPoints` e `Place` de atletas com atributos semelhantes. O professor sugeriu coletar uma amostra dos últimos 5 anos do dataset original e usar 3,5 anos como *training* set, 0,5 anos como *validation* set e o último ano como *test* set.

*Na real eu não entendi se é pra prever a colocação de um atleta de acordo com colocações anteriores deste mesmo atleta ou de acordo com colocações anteriores de atletas semelhantes*

